### Experiment with MongoEngine

The ODM we use to access MongoDB

In [1]:
from dexter.DB import DB, Account, Entry, Transaction

from datetime import date

Open the database:

In [2]:
DB.open('pytest')

Make an account:

In [3]:
acct = Account(name='equity', category='Q')

Save it:

In [4]:
acct.save()

<Account: Account object>

If we open that DB with `mongosh` we should see the account.

```
$ mongosh

test> use foo
switched to db foo

foo> db.account.find()
[
  {
    _id: ObjectId('67c61fa19d0161a19b80469e'),
    name: 'equity',
    group: 'equity'
  }
]
```

It worked!  🎉

### Contents of a Collection

In [5]:
Account.objects

[<Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>, <Account: Account object>]

In [6]:
Account.objects[0]

<Account: Account object>

In [7]:
acct = Account.objects[0]

In [8]:
acct.name

'equity'

### Low Level API

We can also connect to the DB directly to use the `pymongo` library, _e.g._ to get collection names.

After calling `DB.open` we can get a reference to the client and the current database using static vars of the module:

In [9]:
DB.client

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None))

In [10]:
DB.database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest')

In [11]:
db = DB.database

In [12]:
db.account

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'account')

In [13]:
db.account.find_one()

{'_id': ObjectId('67d8a6899a2cc4f5379b0ff1'),
 'name': 'equity',
 'category': 'Q'}

### The Big Picture

Use the high level API when working with data.  MongoEngine converts the documents into objects (which is something we'd be doing ourselves if we didn't use it).

Use the low level API for collective operations: exporting, importing, ...

**NOTE**  It's possible to get a document using the low level API, as shown above, but it will be a `dict`, not a model instance.

### Transactions

In [14]:
t = Transaction(description='hi', comment='aloha')

In [15]:
t.description

'hi'

Nice -- the list fields are initially empty.

In [16]:
t.tags

[]

In [17]:
t.entries

[]

### Entries

In [18]:
p = Entry(uid='xxx', column='credit', date='2025-03-05', amount=1000, account='unknown')

In [19]:
p

<Entry: <En 2025-03-05 unknown -$1000.0>>

In [20]:
p.column

<Column.cr: 'credit'>

In [21]:
p.amount

1000.0

### References

The big test -- can we add that Entry to the transaction?

In [22]:
t.entries.append(p)

In [23]:
t.entries

[<Entry: <En 2025-03-05 unknown -$1000.0>>]

Yes!  🎉

### Misc Commands

In [24]:
db.stats

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')

In [25]:
db.stats.find_one

<bound method Collection.find_one of Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=4, driver=DriverInfo(name='MongoEngine', version='0.29.1', platform=None)), 'pytest'), 'stats')>

In [26]:
db.list_collection_names()

['transaction', 'account', 'entry']

In [27]:
db.command('count','account')

{'n': 13, 'ok': 1.0}

In [28]:
db.command('hello')

{'isWritablePrimary': True,
 'topologyVersion': {'processId': ObjectId('67d82d956572fe0fb182796b'),
  'counter': 0},
 'maxBsonObjectSize': 16777216,
 'maxMessageSizeBytes': 48000000,
 'maxWriteBatchSize': 100000,
 'localTime': datetime.datetime(2025, 3, 19, 23, 57, 4, 834000),
 'logicalSessionTimeoutMinutes': 30,
 'connectionId': 66,
 'minWireVersion': 0,
 'maxWireVersion': 25,
 'readOnly': False,
 'ok': 1.0}

In [29]:
db.command('hostInfo')

{'system': {'currentTime': datetime.datetime(2025, 3, 19, 23, 57, 4, 838000),
  'hostname': 'fiontán.local',
  'cpuAddrSize': 64,
  'memSizeMB': 32768,
  'memLimitMB': 32768,
  'numCores': 10,
  'numCoresAvailableToProcess': 10,
  'numPhysicalCores': 10,
  'numCpuSockets': 1,
  'cpuArch': 'arm64',
  'numaEnabled': False,
  'numNumaNodes': 1},
 'os': {'type': 'Darwin', 'name': 'Mac OS X', 'version': '24.3.0'},
 'extra': {'versionString': 'Darwin Kernel Version 24.3.0: Thu Jan  2 20:24:16 PST 2025; root:xnu-11215.81.4~3/RELEASE_ARM64_T6000',
  'alwaysFullSync': 0,
  'nfsAsync': 0,
  'model': 'MacBookPro18,3',
  'cpuString': 'Apple M1 Pro',
  'pageSize': 16384,
  'scheduler': 'edge'},
 'ok': 1.0}

In [30]:
db.command('ping')

{'ok': 1.0}

### Fetch Transactions

Specify constraints on transactions

In [31]:
Transaction.objects

[<Transaction: <Tr 2024-01-02 yoyodyne -> checking $5000.0 paycheck>>, <Transaction: <Tr 2024-01-02 groceries/mortgage/car/travel -> yoyodyne $5000.0 fill buckets>>, <Transaction: <Tr 2024-02-02 yoyodyne -> checking $5000.0 paycheck>>, <Transaction: <Tr 2024-02-02 groceries/mortgage/car/travel -> yoyodyne $5000.0 fill buckets>>, <Transaction: <Tr 2024-01-05 checking -> car $500.0 car payment>>, <Transaction: <Tr 2024-02-05 checking -> car $500.0 car payment>>, <Transaction: <Tr 2024-01-10 visa -> car $50.0 Shell Oil>>, <Transaction: <Tr 2024-02-26 visa -> car $60.0 Shell Oil>>, <Transaction: <Tr 2024-01-04 checking -> mortgage $1800.0 Rocket Mortgage>>, <Transaction: <Tr 2024-02-04 checking -> mortgage $1800.0 Rocket Mortgage>>, <Transaction: <Tr 2024-01-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-01-21 visa -> groceries $175.0 Safeway>>, <Transaction: <Tr 2024-02-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-02-21 visa -> groceries $75.0 Safeway

In [32]:
Transaction.objects(description='Safeway')

[<Transaction: <Tr 2024-01-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-01-21 visa -> groceries $175.0 Safeway>>, <Transaction: <Tr 2024-02-07 checking -> groceries $75.0 Safeway>>, <Transaction: <Tr 2024-02-21 visa -> groceries $75.0 Safeway>>]

In [33]:
for t in Transaction.objects(description='Safeway'):
    for e in t.entries:
        print(e.date, e.account, e.amount, e.column)

2024-01-07 groceries 75.0 debit
2024-01-07 checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 visa 75.0 credit


In [34]:
for t in Transaction.objects(description='Safeway'):
    print(t.accounts)

{'checking', 'groceries'}
{'visa', 'groceries'}
{'checking', 'groceries'}
{'visa', 'groceries'}


In [35]:
for t in Transaction.objects(description='Safeway'):
    print(t.pamount)

75.0
175.0
75.0
75.0


In [36]:
for t in Transaction.objects(description='Safeway'):
    print(t.pdate, type(t.pdate))

2024-01-07 <class 'datetime.date'>
2024-01-21 <class 'datetime.date'>
2024-02-07 <class 'datetime.date'>
2024-02-21 <class 'datetime.date'>


In [37]:
for t in Transaction.objects(description='Safeway'):
    print(t.originals)

/
big party this weekend/
/
/


In [38]:
lst = list(Transaction.objects(description='Safeway'))

In [39]:
lst[0].to_json()

'{"_id": {"$oid": "67d8a68a9a2cc4f5379b102b"}, "description": "Safeway", "comment": "weekly groceries", "tags": [], "entries": [{"$oid": "67d8a6899a2cc4f5379b1015"}, {"$oid": "67d8a6899a2cc4f5379b1016"}], "pdate": {"$date": 1704585600000}, "pdebit": "groceries", "pcredit": "checking", "pamount": 75.0}'

In [40]:
lst[1].comment

In [41]:
lst[1].pamount

175.0

In [42]:
lst[1].pdate

datetime.date(2024, 1, 21)

In [43]:
list(Transaction.objects(pamount__lt=175.0))

[<Transaction: <Tr 2024-01-10 visa -> car $50.0 Shell Oil>>,
 <Transaction: <Tr 2024-02-26 visa -> car $60.0 Shell Oil>>,
 <Transaction: <Tr 2024-01-07 checking -> groceries $75.0 Safeway>>,
 <Transaction: <Tr 2024-02-07 checking -> groceries $75.0 Safeway>>,
 <Transaction: <Tr 2024-02-21 visa -> groceries $75.0 Safeway>>,
 <Transaction: <Tr 2024-03-10 amex -> household $50.0 Home Depot>>,
 <Transaction: <Tr 2024-03-12 household -> amex $25.0 Home Depot>>]

In [44]:
for t in Transaction.objects(pdate=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

2024-01-02 5000.0 checking yoyodyne
2024-01-02 5000.0 yoyodyne groceries/mortgage/car/travel


In [45]:
for t in Transaction.objects(pdate__lte=date(2024,1,2)):
    print(t.pdate, t.pamount, t.pdebit, t.pcredit)

2024-01-02 5000.0 checking yoyodyne
2024-01-02 5000.0 yoyodyne groceries/mortgage/car/travel


### Operators

In [46]:
for t in Transaction.objects(description__gte='Safeway'):
    print(t.pdate, t.description)

2024-01-02 paycheck
2024-01-02 fill buckets
2024-02-02 paycheck
2024-02-02 fill buckets
2024-01-05 car payment
2024-02-05 car payment
2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


In [47]:
for t in Transaction.objects(description__regex='^S'):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


The operator automatically applies to list elements.

In [48]:
for t in Transaction.objects(description__regex=r'\s'):
    print(t.pdate, t.description, t.pamount)

2024-01-02 fill buckets 5000.0
2024-02-02 fill buckets 5000.0
2024-01-05 car payment 500.0
2024-02-05 car payment 500.0
2024-01-10 Shell Oil 50.0
2024-02-26 Shell Oil 60.0
2024-01-04 Rocket Mortgage 1800.0
2024-02-04 Rocket Mortgage 1800.0
2024-03-10 Home Depot 50.0
2024-03-12 Home Depot 25.0


For compound constraints we need another class from MongoEngine.

In [49]:
from mongoengine.queryset.visitor import Q

In [50]:
for t in Transaction.objects(Q(description__regex=r'^S')):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil
2024-01-07 Safeway
2024-01-21 Safeway
2024-02-07 Safeway
2024-02-21 Safeway


In [51]:
for t in Transaction.objects(Q(description__regex=r'^S') & Q(description__regex=r'\s')):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil


### QuerySet

In [52]:
for a in Account.nominal_accounts:
    print(a.name)

amex
visa
groceries
household
mortgage
car
travel


### Combining Query Elements

In [53]:
q = Q(description__regex=r'^S')

In [54]:
q

Q(**{'description__regex': '^S'})

In [55]:
type(q)

mongoengine.queryset.visitor.Q

In [56]:
p = Q(description__regex=r'\s')

In [57]:
p & q

(Q(**{'description__regex': '\\s'}) & Q(**{'description__regex': '^S'}))

In [58]:
for t in Transaction.objects(p & q):
    print(t.pdate, t.description)

2024-01-10 Shell Oil
2024-02-26 Shell Oil


Create Q object using dictionaries

In [59]:
dct = {'description__regex': r'^S'}

In [60]:
Q(**dct)

Q(**{'description__regex': '^S'})

Can an object have multiple constraints?

In [61]:
dct = {'description__regex': r'^S', 'pamount__gt': 100}

In [62]:
Q(**dct)

Q(**{'description__regex': '^S', 'pamount__gt': 100})

In [63]:
for t in Transaction.objects(Q(**dct)):
    print(t.pdate, t.description, t.pamount)

2024-01-21 Safeway 175.0


Yep!

### Select Method

#### Select Transactions

All transactions:

In [64]:
for t in DB.select(Transaction):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 yoyodyne checking
2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 yoyodyne checking
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-01-05 500.0 checking car
2024-02-05 500.0 checking car
2024-01-10 50.0 visa car
2024-02-26 60.0 visa car
2024-01-04 1800.0 checking mortgage
2024-02-04 1800.0 checking mortgage
2024-01-07 75.0 checking groceries
2024-01-21 175.0 visa groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries
2024-03-10 50.0 amex household
2024-03-12 25.0 household amex


By date:

In [65]:
for t in DB.select(Transaction, date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-21 175.0 visa groceries


In [66]:
for t in DB.select(Transaction, start_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-02-02 5000.0 yoyodyne checking
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-05 500.0 checking car
2024-02-26 60.0 visa car
2024-02-04 1800.0 checking mortgage
2024-01-21 175.0 visa groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries
2024-03-10 50.0 amex household
2024-03-12 25.0 household amex


In [67]:
for t in DB.select(Transaction, end_date=date(2024,1,21)):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 yoyodyne checking
2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-01-05 500.0 checking car
2024-01-10 50.0 visa car
2024-01-04 1800.0 checking mortgage
2024-01-07 75.0 checking groceries
2024-01-21 175.0 visa groceries


By amount:

In [68]:
for t in DB.select(Transaction, amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-07 75.0 checking groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries


In [69]:
lst = DB.select(Transaction, amount=75)

In [70]:
all(t.pamount == 75 for t in lst)

True

In [71]:
for t in DB.select(Transaction, max_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-10 50.0 visa car
2024-02-26 60.0 visa car
2024-01-07 75.0 checking groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries
2024-03-10 50.0 amex household
2024-03-12 25.0 household amex


In [72]:
for t in DB.select(Transaction, min_amount=75):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 yoyodyne checking
2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 yoyodyne checking
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-01-05 500.0 checking car
2024-02-05 500.0 checking car
2024-01-04 1800.0 checking mortgage
2024-02-04 1800.0 checking mortgage
2024-01-07 75.0 checking groceries
2024-01-21 175.0 visa groceries
2024-02-07 75.0 checking groceries
2024-02-21 75.0 visa groceries


By description:

In [73]:
for t in DB.select(Transaction, description = r'^s'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-01-10 50.0 Shell Oil visa car
2024-02-26 60.0 Shell Oil visa car
2024-01-07 75.0 Safeway checking groceries
2024-01-21 175.0 Safeway visa groceries
2024-02-07 75.0 Safeway checking groceries
2024-02-21 75.0 Safeway visa groceries


In [74]:
for t in DB.select(Transaction, comment=r'budget'):
    print(t.pdate, t.pamount, t.description, t.comment, t.pcredit, t.pdebit)

2024-01-02 5000.0 fill buckets 2024 budget groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 fill buckets 2024 budget groceries/mortgage/car/travel yoyodyne


By account:

In [75]:
for t in DB.select(Transaction, debit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-04 1800.0 checking mortgage
2024-02-04 1800.0 checking mortgage


In [76]:
for t in DB.select(Transaction, credit='mortgage'):
    print(t.pdate, t.pamount, t.pcredit, t.pdebit)

2024-01-02 5000.0 groceries/mortgage/car/travel yoyodyne
2024-02-02 5000.0 groceries/mortgage/car/travel yoyodyne


Some random combinations

In [77]:
for t in DB.select(Transaction, description = r'^s', min_amount=100):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-01-21 175.0 Safeway visa groceries


In [78]:
for t in DB.select(Transaction, start_date = date(2024,2,1), credit='visa'):
    print(t.pdate, t.pamount, t.description, t.pcredit, t.pdebit)

2024-02-26 60.0 Shell Oil visa car
2024-02-21 75.0 Safeway visa groceries


#### Select Entries

All entries:

In [79]:
len(DB.select(Entry))

38

In [80]:
for e in DB.select(Entry):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 credit
2024-01-02 yoyodyne 5000.0 debit
2024-01-02 groceries 300.0 credit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-02-02 checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 yoyodyne 5000.0 debit
2024-02-02 groceries 300.0 credit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit
2024-02-05 car 500.0 debit
2024-02-05 checking 500.0 credit
2024-01-10 car 50.0 debit
2024-01-10 visa 50.0 credit
2024-02-26 car 60.0 debit
2024-02-26 visa 60.0 credit
2024-01-04 mortgage 1800.0 debit
2024-01-04 checking 1800.0 credit
2024-02-04 mortgage 1800.0 debit
2024-02-04 checking 1800.0 credit
2024-01-07 groceries 75.0 debit
2024-01-07 checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 checking 75.0

By date:

In [81]:
for e in DB.select(Entry, date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit


In [82]:
for e in DB.select(Entry, start_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-02-02 checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 yoyodyne 5000.0 debit
2024-02-02 groceries 300.0 credit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit
2024-02-05 car 500.0 debit
2024-02-05 checking 500.0 credit
2024-01-10 car 50.0 debit
2024-01-10 visa 50.0 credit
2024-02-26 car 60.0 debit
2024-02-26 visa 60.0 credit
2024-02-04 mortgage 1800.0 debit
2024-02-04 checking 1800.0 credit
2024-01-07 groceries 75.0 debit
2024-01-07 checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 visa 75.0 credit
2024-03-10 household 50.0 debit
2024-03-10 amex 50.0 credit
2024-03-12 household 25.0 credit
2024-03-12 amex 25.0 debit


In [83]:
for e in DB.select(Entry, end_date=date(2024,1,5)):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 credit
2024-01-02 yoyodyne 5000.0 debit
2024-01-02 groceries 300.0 credit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit
2024-01-04 mortgage 1800.0 debit
2024-01-04 checking 1800.0 credit


By amount:

In [84]:
for e in DB.select(Entry, amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 car 900.0 credit
2024-02-02 car 900.0 credit


In [85]:
for e in DB.select(Entry, max_amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 groceries 300.0 credit
2024-01-02 car 900.0 credit
2024-02-02 groceries 300.0 credit
2024-02-02 car 900.0 credit
2024-01-05 car 500.0 debit
2024-01-05 checking 500.0 credit
2024-02-05 car 500.0 debit
2024-02-05 checking 500.0 credit
2024-01-10 car 50.0 debit
2024-01-10 visa 50.0 credit
2024-02-26 car 60.0 debit
2024-02-26 visa 60.0 credit
2024-01-07 groceries 75.0 debit
2024-01-07 checking 75.0 credit
2024-01-21 groceries 175.0 debit
2024-01-21 visa 175.0 credit
2024-02-07 groceries 75.0 debit
2024-02-07 checking 75.0 credit
2024-02-21 groceries 75.0 debit
2024-02-21 visa 75.0 credit
2024-03-10 household 50.0 debit
2024-03-10 amex 50.0 credit
2024-03-12 household 25.0 credit
2024-03-12 amex 25.0 debit


In [86]:
for e in DB.select(Entry, min_amount=900):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 credit
2024-01-02 yoyodyne 5000.0 debit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-02-02 checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 yoyodyne 5000.0 debit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-04 mortgage 1800.0 debit
2024-01-04 checking 1800.0 credit
2024-02-04 mortgage 1800.0 debit
2024-02-04 checking 1800.0 credit


By account:

In [87]:
for e in DB.select(Entry, account='groceries'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 groceries 300.0 credit
2024-02-02 groceries 300.0 credit
2024-01-07 groceries 75.0 debit
2024-01-21 groceries 175.0 debit
2024-02-07 groceries 75.0 debit
2024-02-21 groceries 75.0 debit


By column:

In [88]:
for e in DB.select(Entry, column='credit'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 yoyodyne 5000.0 credit
2024-01-02 groceries 300.0 credit
2024-01-02 mortgage 1800.0 credit
2024-01-02 car 900.0 credit
2024-01-02 travel 2000.0 credit
2024-02-02 yoyodyne 5000.0 credit
2024-02-02 groceries 300.0 credit
2024-02-02 mortgage 1800.0 credit
2024-02-02 car 900.0 credit
2024-02-02 travel 2000.0 credit
2024-01-05 checking 500.0 credit
2024-02-05 checking 500.0 credit
2024-01-10 visa 50.0 credit
2024-02-26 visa 60.0 credit
2024-01-04 checking 1800.0 credit
2024-02-04 checking 1800.0 credit
2024-01-07 checking 75.0 credit
2024-01-21 visa 175.0 credit
2024-02-07 checking 75.0 credit
2024-02-21 visa 75.0 credit
2024-03-10 amex 50.0 credit
2024-03-12 household 25.0 credit


In [89]:
for e in DB.select(Entry, column='debit'):
    print(e.date, e.account, e.amount, e.column)

2024-01-02 checking 5000.0 debit
2024-01-02 yoyodyne 5000.0 debit
2024-02-02 checking 5000.0 debit
2024-02-02 yoyodyne 5000.0 debit
2024-01-05 car 500.0 debit
2024-02-05 car 500.0 debit
2024-01-10 car 50.0 debit
2024-02-26 car 60.0 debit
2024-01-04 mortgage 1800.0 debit
2024-02-04 mortgage 1800.0 debit
2024-01-07 groceries 75.0 debit
2024-01-21 groceries 175.0 debit
2024-02-07 groceries 75.0 debit
2024-02-21 groceries 75.0 debit
2024-03-10 household 50.0 debit
2024-03-12 amex 25.0 debit


### Serializing Objects

In [90]:
import json
from bson.objectid import ObjectId
import datetime

A document created by calling `as_pymongo':

In [91]:
obj = "{'_id': ObjectId('67d5c38ea57aba579a76abe8'), 'description': 'Home Depot', 'comment': '', 'tags': [], 'entries': [ObjectId('67d5c38ea57aba579a76abd7'), ObjectId('67d5c38ea57aba579a76abd8')], 'pdate': datetime.datetime(2024, 3, 12, 0, 0), 'pdebit': 'amex', 'pcredit': 'household', 'pamount': 25.0}"

In [92]:
obj

"{'_id': ObjectId('67d5c38ea57aba579a76abe8'), 'description': 'Home Depot', 'comment': '', 'tags': [], 'entries': [ObjectId('67d5c38ea57aba579a76abd7'), ObjectId('67d5c38ea57aba579a76abd8')], 'pdate': datetime.datetime(2024, 3, 12, 0, 0), 'pdebit': 'amex', 'pcredit': 'household', 'pamount': 25.0}"

In [93]:
obj.replace("'",'"')

'{"_id": ObjectId("67d5c38ea57aba579a76abe8"), "description": "Home Depot", "comment": "", "tags": [], "entries": [ObjectId("67d5c38ea57aba579a76abd7"), ObjectId("67d5c38ea57aba579a76abd8")], "pdate": datetime.datetime(2024, 3, 12, 0, 0), "pdebit": "amex", "pcredit": "household", "pamount": 25.0}'

In [94]:
rec = f'["accounts", {obj.replace("'",'"')}]'

In [95]:
rec

'["accounts", {"_id": ObjectId("67d5c38ea57aba579a76abe8"), "description": "Home Depot", "comment": "", "tags": [], "entries": [ObjectId("67d5c38ea57aba579a76abd7"), ObjectId("67d5c38ea57aba579a76abd8")], "pdate": datetime.datetime(2024, 3, 12, 0, 0), "pdebit": "amex", "pcredit": "household", "pamount": 25.0}]'

In [96]:
json.loads(rec)

JSONDecodeError: Expecting value: line 1 column 22 (char 21)

In [98]:
collection = DB.database["foo"]

In [99]:
collection.insert_one(obj)

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping

In [97]:
eval(obj)

{'_id': ObjectId('67d5c38ea57aba579a76abe8'),
 'description': 'Home Depot',
 'comment': '',
 'tags': [],
 'entries': [ObjectId('67d5c38ea57aba579a76abd7'),
  ObjectId('67d5c38ea57aba579a76abd8')],
 'pdate': datetime.datetime(2024, 3, 12, 0, 0),
 'pdebit': 'amex',
 'pcredit': 'household',
 'pamount': 25.0}

In [102]:
collection.insert_one(eval(obj))

InsertOneResult(ObjectId('67d5c38ea57aba579a76abe8'), acknowledged=True)

In [101]:
collection = DB.database["bar"]